In [1]:
%matplotlib inline
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
from shapely.geometry import mapping
sys.path.append(r'E:/gitlab/geoloc2/Detecciondeterrenos')
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r'C:/Users/ASUS/Inteligencia_Artificial/yolov7')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import time
import datetime

In [2]:
# import warnings
# warnings.filterwarnings('ignore')
# import pandas as pd
# import math
# from shapely.geometry import Polygon
# import cv2
# from clasificacion_chinchetas import *
# import tqdm
# import numpy as np
# from osgeo import gdal
# import rasterio
# import geopandas as gpd
# import rasterio.mask
# import sys
# from shapely.geometry import mapping
# from shapely import geometry
# import cv2
# import numpy as np

## Raster y shape deteccion

In [3]:
raster=r"E:\rastermunicpial\iztpaan_sal.tif"
src=rasterio.open(raster)

deteccion=r"E:\rastermunicpial\iztapan_sal\Ixtapan_casas_mun_Z.shp"
deteccion_shp=gpd.read_file(deteccion)


In [4]:
deteccion_shp=deteccion_shp.dropna().reset_index(drop=True)
angulo_manzana=[]
for manzana in range(len(deteccion_shp)):
    proyecciones1=mapping(deteccion_shp['geometry'][manzana]).get('coordinates')
    angulos=[]
    d=[]
    poly=pd.DataFrame(proyecciones1[0])
    for point in range(1,len(poly)):
        d.append(((poly[1][point]-poly[1][point-1])**2+(poly[0][point]-poly[0][point-1])**2))
#         angulos.append(math.atan(((poly[1][point]-poly[1][point-1])/(poly[0][point]-poly[0][point-1])))*180/math.pi)
#     angulo_manzana.append(angulos[d.index(max(d))])
# deteccion_shp['angulo']=angulo_manzana
deteccion_shp1=ampliar_shape(deteccion_shp,factor_ampliacion=1.5)

## Clasificar casas y/o terrenos (Alexnet)

##### Pesos y diccionario Alexnet

In [5]:
# weights=r'C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint/best_mod11.pth'
# num_classes=5
# diciconario={0: 'casas', 1: 'en_construccion', 2: 'establecimiento', 3: 'multivivienda', 4: 'terreno_baldio'}
# model_class=alexnet(weights=weights,num_classes=num_classes,idx_to_class=diciconario)


# weights=r"C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint\best_modell11.pth"
# num_classes=6
# diciconario={0: 'carros', 1: 'casas', 2: 'en_construccion', 3: 'establecimiento', 4: 'multivivienda', 5: 'terreno_baldio'}
# model_class=alexnet(weights=weights,num_classes=num_classes,idx_to_class=diciconario)

#### Clasificación

In [7]:
%matplotlib
import tqdm
import cv2
geometry=[]
status=[]
for i,polygo in tqdm.tqdm(enumerate(deteccion_shp1["geometry"][3:]),total=len(deteccion_shp1)):
    det="a"
    try:
        shapes=[mapping(polygo)]
    except:
        continue
#     out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    imagen=cv2.cvtColor(np.moveaxis(array,0,-1),cv2.COLOR_RGB2BGR)
    
    img =imagen.copy()
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    gray=(normalizar_array(gray)*2-1)*255
    #plt.imshow(gray)
    #plt.show()
    y=gray.copy()
    y[y<5]=0
    y[y>=5]=255
    gray=cv2.erode(y,(6,6))
    # cv2_imshow("y",np.array(np.hstack([gray,imagen[:,:,0]]),dtype="uint8"))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=np.array(img[:,:,1],dtype=float)**2
    h,w= img.shape
    yc,xc=int(h/2),int(w/2)
    thres=img.copy()
    u=190
    thres[thres<u**2]=0
    thres[thres>=u**2]=255
    thres=np.array(thres,dtype="uint8")#_,thres=cv2.threshold(img,173,255,1)
    f=1.25
    kernel = np.array([[-1,-1,1],[-1,1,-1],[1,-1,-1]])
    kernel=np.array([[1,1,1],[0,0,1],[0,0,1]])
    dst = cv2.filter2D(thres,-1,kernel)
    cont,hist=cv2.findContours(dst,2,1)
    res=pd.DataFrame({"c":cont,"s":[cv2.contourArea(cnt) for cnt in cont]})
    res.sort_values(by="s",ascending=False,inplace=True)
    res=res.reset_index(drop=True)
    try:
        cnt=res.c.values[0]
    except:
        geometry.append(polygo)
        status.append("error")
        continue
    if res.loc[0,"s"]<.5*polygo.area:
        geometry.append(polygo)
        status.append("area_small")
        continue
    epsilon = 0.02*cv2.arcLength(cnt,True)
    approx = cv2.approxPolyDP(cnt,epsilon,True)
    cnt=approx
    s=np.zeros_like(dst)
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    coords=shapes[0].get("coordinates")[0]
    left,bottom=np.min(coords,axis=0)
    right,top=np.max(coords,axis=0)
    height=imagen.shape[0]
    width=imagen.shape[1]
    # cv2.drawContours(s,[box],0,(127),2)
    # cv2.drawContours(s,[cnt],-1,(255),-1)
    geometry.append(Polygon([(map_d(x[0],0,width,left,right),map_d(x[1],height,0,bottom,top)) for x in box]))
    status.append("correcto")
    # cv2_imshow("s",s)
    # plt.show()
    #break
    # gray = np.float32(gray)
    # dst = cv.cornerHarris(gray,2,3,0.04)
    # dst[dst<=0]=0
    # dst=dst
    # sa=dst.copy()
    # sa=sa.reshape(-1)
    # sa.sort()
    # box=np.moveaxis(np.where(dst>=sa[-4]),0,-1)
    # # print(box)
    # box=np.stack([box[:,1],box[:,0]],axis=-1)
    # # box
    # # cv2.drawContours(img,[box],0,(0,0,255),2)
    # cv2_imshow("s",img)
#     rows,cols = img.shape[:2]
#     [vx,vy,x,y] = cv.fitLine(box, cv.DIST_L2,0,0.01,0.01)
#     lefty = int((-x*vy/vx) + y)
#     righty = int(((cols-x)*vy/vx)+y)
#     cv.line(img,(cols-1,righty),(0,lefty),(0,255,0),2)
#     cv2_imshow("s",img)
#     # four_images=[array[2],array[1],array[0]]
#     # stacked_images = np.stack(four_images, axis=-1)
    
# #     imagen_1=rotate_image(stacked_images,deteccion_shp["angulo"][i],reshape=False)
# #     clase,imagen=model_class.predict_image(imagen_1)
    
# #     cv2.|imshow("im",imagen)
# #     cv2.waitKey()
# #     cv2.destroyAllWindows()
#     deteccion_shp.loc[i,"clase_n"]=clase

Using matplotlib backend: Qt5Agg


100%|█████████▉| 27145/27148 [02:34<00:00, 176.11it/s]


In [ ]:
gpd.GeoDataFrame({"status":status},geometry=geometry,crs=3857).to_file(r"E:\rastermunicpial/test_nuevarotacion.shp")


In [63]:
res.loc[i,"c"]

#  box_l.append(Polygon([(map_d(x[0],0,width,width_min,width_max),map_d(x[1],heigth,0,heigth_min,height_max)) for x in box]))
coords=shapes[0].get("coordinates")[0]
left,bottom=np.min(coords,axis=0)
right,top=np.max(coords,axis=0)


height=imagen.shape[0]
width=imagen.shape[1]
left,bottom,right,top,height,width


           

(-11104562.72925238,
 2138333.896826212,
 -11104537.723022468,
 2138355.8741578227,
 56,
 63)

In [78]:
geomtry=(Polygon([(map_d(x[0],0,width,left,right),map_d(x[1],height,0,bottom,top)) for x in box]))

'POLYGON ((-11104560.347706674 2138343.708134967, -11104546.852281008 2138351.9496343206, -11104542.48611388 2138345.6703967177, -11104555.981539546 2138337.0364450132, -11104560.347706674 2138343.708134967))'

In [77]:
deteccion_shp

,clase_dete,conf,area,Z,geometry,centroid
0,establecimiento,0.24460,90.506979,3.000000,"POLYGON ((-11104937.137 2135063.112, -11104927...",POINT (-11104931.480 2135059.438)
1,casas,0.41648,90.506979,4.531915,"POLYGON ((-11104928.147 2135064.029, -11104918...",POINT (-11104922.490 2135060.355)
2,terreno_baldio,0.22695,63.354885,3.000000,"POLYGON ((-11104715.690 2134795.864, -11104708...",POINT (-11104712.181 2134791.350)
3,establecimiento,0.84506,162.912562,2.987616,"POLYGON ((-11104560.229 2138343.579, -11104545...",POINT (-11104550.226 2138344.885)
4,establecimiento,0.78622,1407.886334,4.000000,"POLYGON ((-11104534.396 2138602.027, -11104482...",POINT (-11104503.661 2138601.459)
...,...,...,...,...,...,...
27143,establecimiento,0.40038,193.081554,1.948052,"POLYGON ((-11090130.950 2135756.937, -11090120...",POINT (-11090121.949 2135752.519)
27144,establecimiento,0.24816,253.419540,4.000000,"POLYGON ((-11090098.783 2135837.148, -11090087...",POINT (-11090087.803 2135833.971)
27145,casas,0.39297,80.450648,2.000000,"POLYGON ((-11090094.895 2135875.313, -11090086...",POINT (-11090088.483 2135875.015)
27146,establecimiento,0.46477,543.041872,8.366973,"POLYGON ((-11090103.060 2135832.798, -11090086...",POINT (-11090086.995 2135827.739)


In [38]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
# img=cv2.imread("Alarms/malavi/industria~2.png")
# img=cv2.resize(img,(60,60))
# img=cv2.resize(img,(100,100))
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img=np.array(img[:,:,1],dtype=float)**2
h,w= img.shape
yc,xc=int(h/2),int(w/2)
thres=img.copy()
u=190
thres[thres<u**2]=0
thres[thres>=u**2]=255
thres=np.array(thres,dtype="uint8")#_,thres=cv2.threshold(img,173,255,1)
f=1.25
kernel = np.array([[-1,-1,1],[-1,1,-1],[1,-1,-1]])
kernel=np.array([[1,1,1],[0,0,1],[0,0,1]])
dst = cv2.filter2D(thres,-1,kernel)
cont,hist=cv2.findContours(dst,2,1)
res=pd.DataFrame({"c":cont,"s":[cv2.contourArea(cnt) for cnt in cont]})
res.sort_values(by="s",ascending=False,inplace=True)
cnt=res.c.values[0]
epsilon = 0.02*cv2.arcLength(cnt,True)
approx = cv2.approxPolyDP(cnt,epsilon,True)
cnt=approx
s=np.zeros_like(dst)
rect = cv2.minAreaRect(cnt)
box = cv2.boxPoints(rect)
box = np.int0(box)
cv2.drawContours(s,[box],0,(127),2)
cv2.drawContours(s,[cnt],-1,(255),-1)
plt.imshow(s)
plt.show()
print(cont)

(array([[[39, 11]],

       [[38, 12]],

       [[37, 13]],

       [[36, 13]],

       [[35, 14]],

       [[34, 15]],

       [[33, 15]],

       [[32, 16]],

       [[31, 17]],

       [[30, 18]],

       [[29, 18]],

       [[28, 18]],

       [[27, 19]],

       [[26, 19]],

       [[25, 20]],

       [[24, 21]],

       [[23, 21]],

       [[22, 22]],

       [[21, 22]],

       [[20, 23]],

       [[19, 24]],

       [[18, 24]],

       [[18, 25]],

       [[18, 26]],

       [[18, 27]],

       [[17, 28]],

       [[16, 27]],

       [[16, 26]],

       [[15, 27]],

       [[14, 27]],

       [[13, 28]],

       [[12, 29]],

       [[12, 30]],

       [[12, 31]],

       [[12, 32]],

       [[12, 33]],

       [[13, 34]],

       [[13, 35]],

       [[13, 36]],

       [[13, 37]],

       [[12, 38]],

       [[11, 37]],

       [[11, 36]],

       [[11, 37]],

       [[11, 38]],

       [[12, 38]],

       [[13, 38]],

       [[14, 37]],

       [[15, 37]],

       [[16, 38]],


In [17]:
y=gray.copy()
y[y<200]=0
y=cv2.erode(y,(6,6))
cv2_imshow("y",y)

In [10]:
pd.DataFrame(gray).to_clipboard()

In [1]:
import numpy as np
import cv2 as cv
filename = 'chessboard.png'
img =imagen.copy()
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
gray = np.float32(gray)
dst = cv.cornerHarris(gray,2,3,0.04)
dst[dst<=0]=0
dst=dst**2
sa=dst.copy()
sa=sa.reshape(-1)
sa.sort()
box=np.moveaxis(np.where(dst>=sa[-4]),0,-1)
print(box)
box=np.stack([box[:,1],box[:,0]],axis=-1)
# box
# cv2.drawContours(img,[box],0,(0,0,255),2)
# cv2_imshow("s",img)
rows,cols = img.shape[:2]
[vx,vy,x,y] = cv.fitLine(box, cv.DIST_L2,0,0.01,0.01)
lefty = int((-x*vy/vx) + y)
righty = int(((cols-x)*vy/vx)+y)
cv.line(img,(cols-1,righty),(0,lefty),(0,255,0),2)
cv2_imshow("s",img)
#result is dilated for marking the corners, not important
# dst = cv.dilate(dst,None)
# Threshold for an optimal value, it may vary depending on the image.
# img[dst>0.01*dst.max()]=
img[dst>0.05*dst.max()]=[0,0,255]
cv2_imshow('dst',img)
# if cv.waitKey(0) & 0xff == 27:
#     cv.destroyAllWindows()

NameError: name 'imagen' is not defined

In [ ]:
%matplotlib

xp, yp,_= img.shape

x = np.arange(0, xp, 1)
y = np.arange(0, yp, 1)
Y, X = np.meshgrid(y, x)
fig=plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(X,Y,dst,c=np.random.randint(0,255,len(dst.reshape(-1)))/255)

In [ ]:
sa=dst.copy()
sa=sa.reshape(-1)
sa.sort()
box=np.moveaxis(np.where(dst>=sa[-4]),0,-1)
print(box)
box=np.stack([box[:,1],box[:,0]],axis=-1)
# box
# cv2.drawContours(img,[box],0,(0,0,255),2)
# cv2_imshow("s",img)
rows,cols = img.shape[:2]
[vx,vy,x,y] = cv.fitLine(box, cv.DIST_L2,0,0.01,0.01)
lefty = int((-x*vy/vx) + y)
righty = int(((cols-x)*vy/vx)+y)
cv.line(img,(cols-1,righty),(0,lefty),(0,255,0),2)
cv2_imshow("s",img)

## Clasificar chinchetas (Matching template)

#### Plantillas

In [ ]:
# No borrar path compañeros

path_ruben = r'C:\Users\ruben\Desktop\GEM\Geo\Bases\Plantillas'
path_hector = '/home/hector/Documentos/Infis/Geo/Data/Plantillas'
dict_gris = get_dict_plantilla_gris(path_ruben,True,(30,30))

#### Clasificación

In [ ]:
import tqdm
import cv2
dict_gris = get_dict_plantilla_gris(path_ruben,True,(30,30))
for i,polygo in tqdm.tqdm(enumerate(deteccion_shp1["geometry"]),total=len(deteccion_shp1)):
    det="a"
    try:
        shapes=[mapping(polygo)]
    except:
        continue
#     out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    four_images=[array[2],array[1],array[0]]
    stacked_images = np.stack(four_images, axis=-1)
    
    imagen_1=rotate_image(stacked_images,deteccion_shp["angulo"][i],reshape=False)

    # clasificacion
    clase_aux, n_aux, umbral_aux  = iter_umbral_fn(imagen_1, dict_gris,n=100,salto_n=10,
                                                    umbral=0.94,min_umbral=0.8)
    
    # Revisamos la mayor confusion que se da ... areas_verdes ~ establecimiento_google
    if clase_aux in ['a_verdes','establecimiento_google']:
        clase_aux = areasv_vs_estabgoogle(imagen_1, dict_gris)    
  
    deteccion_shp.loc[i,"clase_chincheta"] = clase_aux
    deteccion_shp.loc[i,"umbral"] = umbral_aux
    deteccion_shp.loc[i,"n_resize"] = n_aux